## AGENDA:


## Created Twitter Dev account and retreived LIVE tweets 
## Scrapping the tweets from twitter
## Cleaned the tweets of any stopwords, #, @, lemmetization, punctuations, smiley's, repetitions, etc.
## Finally separate CSV files were created for each political party

In [11]:
########################################Sentiment Analysis on Political parties using Twitter Data #########################
##############                                   Author : Tegbir Singh Sidhu, Rupinder Sandhu                        ########
##############                                                                                                      ########
#############                                   Date: 2nd December, 2019                                            ########
############################################################################################################################

import os
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import nltk 
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
 
#########################################################################################################

#passing authentication keys for the twitter API

consumer_key = 'O50ppVwfOVht4F0WkmDjp9tcF'
consumer_secret = '4t9BSNG3XWyJOQM1TvHWcQT1aDg5VLHLRWHbnkr7J4VKykFxaZ'
access_key= '1048701354829631493-nvNsvEG2Jm1Dnv6JgFABqSr2MlByD1'
access_secret = 'Rbc1XohWaBDYYP7SMQSPIanRtEXFoJnM4vhKHOHan0zbK'

#supplying the keys into OAuthHandler of tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

########################################################################################################

#Designing the CSV files
#creating paths to store the received data from twitter
ndp_tweets = "C:/Users/Tegbir/Desktop/MyPython/ndp.csv"
liberal_tweets = "C:/Users/Tegbir/Desktop/MyPython/liberal.csv"
conservative_tweets = "C:/Users/Tegbir/Desktop/MyPython/conservative.csv"
green_tweets = "C:/Users/Tegbir/Desktop/MyPython/green.csv"
Bloc_tweets = "C:/Users/Tegbir/Desktop/MyPython/bloc.csv"
Peoplesparty_tweets = "C:/Users/Tegbir/Desktop/MyPython/peoplesparty.csv"

#columns of the csv file
fields = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
'favorite_count', 'retweet_count', 'original_author',   'possibly_sensitive', 'hashtags',
'user_mentions']

#################################################################################################################

#Defining data sets for tranformations
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

#making bigregex for replacing happy emotions with word happy
big_regex_happy = re.compile('|'.join(map(re.escape, emoticons_happy)))
big_regex_sad = re.compile('|'.join(map(re.escape, emoticons_sad)))

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

Extra_Symbols ="*/."

#calling the WordNetLemmatizer method from the corpus of NLTK package.
wordnet_lemmatizer = WordNetLemmatizer()


#####################################################################################################

def TweetsClean (tweet):
    
    #this will get the stopwords from NLTK.
    stop_words = set(stopwords.words('english'))
    
    #need to remove : and Ä¶ 
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r',Ä¶', '', tweet)
    tweet = re.sub(r'â€™','', tweet)
    tweet = re.sub(r'[â€¦]','', tweet)
    tweet = re.sub(r'[â€¦"\~``*.]','', tweet)
    tweet = tweet.replace("'","")
    tweet = tweet.replace('"',"")
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    
    #substitute the emoticons with their associated feeling:
    tweet = big_regex_happy.sub('happy', tweet)
    tweet = big_regex_sad.sub('sad', tweet)
    
    #removing the emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    
    #tokenizing the text in the twitter data
    word_tokens = word_tokenize(tweet)
    
    #filtering using stopwords and punctuations
    #filter using NLTK library and append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words and string.punctuation]
    filtered_tweet = []
    
    for w in word_tokens:
        if w not in stop_words and w not in string.punctuation and w not in Extra_Symbols:
            filtered_tweet.append(wordnet_lemmatizer.lemmatize(w, pos='a'))
    return' '.join(filtered_tweet)
    
###############################################################################################################
#Now in this part we will write the output to the csv files

def write_tweets(keyword, file):
    if os.path.exists(file):
        df = pd.read_csv(file, header = 0)
    else:
        df = pd.DataFrame(columns = fields)
    for page in tweepy.Cursor(api.search, q=keyword, count = 200, include_rts = False).pages(50):
        
        #response from twitter is in json format
        for status in page:
            new_entry = []
            status = status._json
            if status['lang'] != 'en':
                continue

            #Now we will call the methods

            #method for basic preprocessing:
            clean_text = p.clean(status['text'])
            
            
            #calling the data cleaning method:
            filtered_tweet = TweetsClean(clean_text)
            filtered_tweet.lower()
            
            #using textblob to perform sentiment check.
            blob = TextBlob(filtered_tweet.lower())
            Sentiment = blob.sentiment
            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
            new_entry += [status['id'], status['created_at'],status['source'],status['text'], 
                          filtered_tweet, Sentiment, polarity, subjectivity, status['lang'], 
                          status['favorite_count'], status['retweet_count']]
            new_entry.append(status['user']['screen_name'])
            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)
            
            #checking for hashtags
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            #checking for mentions
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)
            
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            #writing to a datafram csvFile using the utf-8 encoding
            csvFile = open(file, 'a' ,encoding='utf-8')
    #writing the dataframe to csvfiels 
    df.to_csv(csvFile, mode='a', columns=fields, index=False, encoding="utf-8")
    
######################################################################################################################
    
#declaring the keywords for our twitter data
ndp_keywords = '#ndp OR #jagmeetsingh OR #ptndp OR #theJagmeetSingh '
Conservative_keywords = '#AndrewScheer OR #conservative OR #ConservativeParty OR #cpc OR #pttory'
GreenParty_keywords = '#greenparty OR #greenparty OR #gpc OR #ptgreen'
Liberal_keywords = '#liberalparty OR #justintrudeau OR #lpc OR #ptlib'
Bloc_keywords = '#bloc OR #blocQuebec OR #ptbloc'
PeoplesParty_keywords = '#Peoplesparty OR #PeoplesPartyCanada'

#################################################################################################################

#calling the mainfunction using our keywords and the ouput filepaths
write_tweets(ndp_keywords,  ndp_tweets)
write_tweets(Conservative_keywords, conservative_tweets)
write_tweets(GreenParty_keywords, green_tweets)
write_tweets(Liberal_keywords, liberal_tweets)
write_tweets(Bloc_keywords, Bloc_tweets)
write_tweets(PeoplesParty_keywords, Peoplesparty_tweets)

#########################################################################################################


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tegbir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tegbir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: module 'preprocessor' has no attribute 'clean'